In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

In [1]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall:

In [3]:
import os
from pathlib import Path
import shutil
import yaml
import random

import torch
from ultralytics import YOLO    # <-- only this is required

from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torch.nn as nn
import torch.optim as optim


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
from pathlib import Path
import shutil
import os

# ✅ Define paths
ROOT = Path("/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset - v2/The Welding Defect Dataset - v2")
WORK = Path("/kaggle/working/welding-data")

# ✅ Recreate working directory (optional: clean start)
if WORK.exists():
    shutil.rmtree(WORK)
os.makedirs(WORK, exist_ok=True)

# ✅ Copy train/valid/test folders (images + labels)
for split in ("train", "valid", "test"):
    src_img = ROOT / split / "images"
    src_lbl = ROOT / split / "labels"
    
    dst_img = WORK / split / "images"
    dst_lbl = WORK / split / "labels"
    
    shutil.copytree(src_img, dst_img)
    shutil.copytree(src_lbl, dst_lbl)

print("✅ Data successfully copied to working directory!")


✅ Data successfully copied to working directory!


In [4]:
updated_yaml = """
train: /kaggle/working/welding-data/train/images
val: /kaggle/working/welding-data/valid/images
test: /kaggle/working/welding-data/test/images

nc: 3
names: ['Bad Weld', 'Good Weld', 'Defect']
"""

with open("/kaggle/working/welding-data/data.yaml", "w") as f:
    f.write(updated_yaml.strip())

print("✅ data.yaml updated and saved in working directory.")


✅ data.yaml updated and saved in working directory.


In [5]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # or yolov8s.pt, yolov8m.pt etc.

model.train(data="/kaggle/working/welding-data/data.yaml", epochs=20, imgsz=416, batch=8)


100%|██████████| 6.25M/6.25M [00:00<00:00, 111MB/s]


Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/welding-data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

100%|██████████| 755k/755k [00:00<00:00, 28.6MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 101MB/s]


...

                   all        283        802      0.516       0.59       0.56       0.33



20 epochs completed in 0.103 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  8.43it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        283        802      0.515      0.591       0.56      0.331
              Bad Weld        141        194      0.544      0.753       0.71      0.452
             Good Weld        175        335      0.547      0.815      0.717      0.437
                Defect        128        273      0.454      0.205      0.255      0.103
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aeabd7e9910>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [6]:
from pathlib import Path
import os
import shutil
import random

SRC = Path("/kaggle/working/welding-data")
DST = Path("/kaggle/working/classification_data")
os.makedirs(DST, exist_ok=True)

categories = ["Bad Weld", "Good Weld", "Defect"]

def parse_labels(label_file):
    # Read YOLO label file and get class indices
    with open(label_file, 'r') as f:
        lines = f.readlines()
    return list(set(int(line.split()[0]) for line in lines if line.strip()))

for split in ['train', 'valid', 'test']:
    split_img_dir = SRC / split / "images"
    split_lbl_dir = SRC / split / "labels"

    for img_file in split_img_dir.glob("*.jpg"):
        label_file = split_lbl_dir / (img_file.stem + ".txt")

        if not label_file.exists():
            continue

        classes = parse_labels(label_file)
        for cls in classes:
            class_name = categories[cls]

            dst_folder = DST / split / class_name
            dst_folder.mkdir(parents=True, exist_ok=True)

            dst_path = dst_folder / img_file.name
            shutil.copy(img_file, dst_path)


In [7]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'valid': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
}

from torchvision.datasets import ImageFolder

data_dir = "/kaggle/working/classification_data"
image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'valid']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True)
               for x in ['train', 'valid']}

class_names = image_datasets['train'].classes
print("Classes:", class_names)


Classes: ['Bad Weld', 'Defect', 'Good Weld']


In [8]:
# Re-import required modules
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from pathlib import Path


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)

# Replace the last FC layer for 3 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 162MB/s]


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 5

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    print(f"Train Loss: {running_loss:.4f}, Accuracy: {100 * correct / total:.2f}%")


Epoch 1/5
Train Loss: 81.3431, Accuracy: 46.94%
Epoch 2/5
Train Loss: 67.4455, Accuracy: 53.06%
Epoch 3/5
Train Loss: 64.6972, Accuracy: 55.31%
Epoch 4/5
Train Loss: 60.9092, Accuracy: 56.78%
Epoch 5/5
Train Loss: 60.0056, Accuracy: 57.71%


In [11]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in dataloaders['valid']:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


Validation Accuracy: 53.38%


In [12]:
import os
from pathlib import Path
import shutil
import yaml
import random
import collections # To count class frequencies

import torch
# from ultralytics import YOLO # Uncomment if you also need to run YOLO training here, otherwise not strictly needed for classification

from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler # Import for learning rate scheduler

# --- Configuration and Setup ---
# Define paths
ROOT = Path("/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset - v2/The Welding Defect Dataset - v2")
WORK = Path("/kaggle/working/welding-data")
CLASSIFICATION_DATA_DIR = Path("/kaggle/working/classification_data")

# Recreate working directory (optional: clean start)
# This ensures a fresh start if you run the notebook multiple times
if WORK.exists():
    shutil.rmtree(WORK)
os.makedirs(WORK, exist_ok=True)

# Copy train/valid/test folders (images + labels) from input to working directory
# This is necessary because Kaggle input datasets are read-only.
print("Copying data to working directory...")
for split in ("train", "valid", "test"):
    src_img = ROOT / split / "images"
    src_lbl = ROOT / split / "labels"
    
    dst_img = WORK / split / "images"
    dst_lbl = WORK / split / "labels"
    
    shutil.copytree(src_img, dst_img)
    shutil.copytree(src_lbl, dst_lbl)

print("✅ Data successfully copied to working directory!")

# Update data.yaml for YOLO (if you still plan to run YOLO training)
# This part is for the YOLO object detection model, not directly used by the classification model,
# but kept for completeness as per your original request.
updated_yaml = """
train: /kaggle/working/welding-data/train/images
val: /kaggle/working/welding-data/valid/images
test: /kaggle/working/welding-data/test/images

nc: 3
names: ['Bad Weld', 'Good Weld', 'Defect']
"""
with open("/kaggle/working/welding-data/data.yaml", "w") as f:
    f.write(updated_yaml.strip())
print("✅ data.yaml updated and saved in working directory.")

# --- Prepare data for classification ---
# Create the base directory for classification data
os.makedirs(CLASSIFICATION_DATA_DIR, exist_ok=True)

# Define the category names based on your dataset's class indices
categories = ["Bad Weld", "Good Weld", "Defect"]

def parse_labels(label_file):
    """Reads YOLO label file and returns unique class indices found."""
    with open(label_file, 'r') as f:
        lines = f.readlines()
    return list(set(int(line.split()[0]) for line in lines if line.strip()))

# Distribute images into classification folders based on their YOLO labels
# Also, collect class counts for weighted loss calculation from the training set
train_class_counts = collections.defaultdict(int)

print("Preparing classification data...")
for split in ['train', 'valid', 'test']:
    split_img_dir = WORK / split / "images"
    split_lbl_dir = WORK / split / "labels"

    # Iterate through each image file in the current split's image directory
    for img_file in split_img_dir.glob("*.jpg"):
        # Construct the corresponding label file path
        label_file = split_lbl_dir / (img_file.stem + ".txt")

        # Skip if no label file exists for the image
        if not label_file.exists():
            continue

        # Parse the class indices from the label file
        classes = parse_labels(label_file)
        # Skip if the label file is empty or contains no valid classes
        if not classes: 
            continue

        # For classification, an image might contain multiple defect types.
        # We copy the image into each class folder it belongs to.
        for cls_idx in classes:
            # Ensure the class index is within the valid range
            if 0 <= cls_idx < len(categories): 
                class_name = categories[cls_idx]
                # Define the destination folder for the current class and split
                dst_folder = CLASSIFICATION_DATA_DIR / split / class_name
                # Create the destination folder if it doesn't exist
                dst_folder.mkdir(parents=True, exist_ok=True)
                # Define the full destination path for the image
                dst_path = dst_folder / img_file.name
                # Copy the image to the destination
                shutil.copy(img_file, dst_path)
                
                # If it's a training image, count its class occurrence for weighting
                if split == 'train':
                    train_class_counts[cls_idx] += 1
            else:
                print(f"Warning: Class index {cls_idx} out of range for {img_file.name}")

print("✅ Classification data prepared and organized.")
print("Training class counts:", dict(train_class_counts))

# --- Calculate Class Weights for Weighted Loss ---
# Convert counts from defaultdict to a list, ordered by class index (0, 1, 2)
class_counts_list = [train_class_counts[i] for i in range(len(categories))]
print("Class counts (ordered by index):", class_counts_list)

# Calculate inverse frequencies for weighting.
# A common approach is max_count / count_i, which gives higher weights to rarer classes.
total_samples = sum(class_counts_list)
if total_samples == 0:
    print("Error: No training samples found for class weight calculation. Using equal weights.")
    class_weights = torch.ones(len(categories)) # Default to equal weights if no samples
else:
    max_count = max(class_counts_list)
    # Calculate weights. Add a small epsilon to avoid division by zero if a class has 0 samples,
    # though with `if count > 0 else 1.0` this is handled.
    class_weights = torch.tensor([max_count / count if count > 0 else 1.0 for count in class_counts_list], dtype=torch.float32)
    
    # Optional: Normalize weights so they sum to the number of classes, or just sum to 1.
    # This might not be strictly necessary for CrossEntropyLoss, but can make interpretation easier.
    # class_weights = class_weights / class_weights.sum() * len(categories) 

print("Calculated Class Weights (higher values for rarer classes):", class_weights)


# --- Enhanced Data Transforms for Classification ---
# These transformations augment the training data to improve model generalization.
# Validation and test transforms are deterministic for consistent evaluation.
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)), # Randomly crop and resize, with a scale variation
        transforms.RandomHorizontalFlip(),                   # Randomly flip the image horizontally
        transforms.RandomRotation(20),                       # Randomly rotate by up to 20 degrees
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.15), # Randomly change color properties
        transforms.ToTensor(),                               # Convert PIL Image to PyTorch Tensor (scales to [0,1])
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Normalize with ImageNet mean and std
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),            # Resize the smaller edge to 256
        transforms.CenterCrop(224),        # Crop the central 224x224 region
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([ # Use the same transforms as validation for consistency
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

Copying data to working directory...
✅ Data successfully copied to working directory!
✅ data.yaml updated and saved in working directory.
Preparing classification data...
✅ Classification data prepared and organized.
Training class counts: {1: 1019, 2: 776, 0: 785}
Class counts (ordered by index): [785, 1019, 776]
Calculated Class Weights (higher values for rarer classes): tensor([1.2981, 1.0000, 1.3131])


In [13]:
# --- Load Datasets and Create DataLoaders ---
from torchvision.datasets import ImageFolder

# Create ImageFolder datasets for train, valid, and test splits
image_datasets = {x: ImageFolder(os.path.join(CLASSIFICATION_DATA_DIR, x), data_transforms[x])
                  for x in ['train', 'valid', 'test']} 

# Create DataLoaders for train and valid splits (shuffled for training)
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2) 
               for x in ['train', 'valid']}

# Create DataLoader for the test split (no shuffling needed for evaluation)
dataloaders['test'] = DataLoader(image_datasets['test'], batch_size=32, shuffle=False, num_workers=2)

# Get dataset sizes and class names for logging
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid', 'test']}
class_names = image_datasets['train'].classes
print("Classes detected by ImageFolder:", class_names)
print("Dataset sizes:", dataset_sizes)

# --- Model Definition and Fine-tuning Setup ---
# Set device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load a pre-trained ResNet18 model with default (best available) weights
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# 1. Freeze all parameters in the base model initially
# This prevents the pre-trained convolutional layers from being updated initially
# while the new classification head learns.
print("Freezing all base model parameters...")
for param in model.parameters():
    param.requires_grad = False

# 2. Replace the last fully connected layer (classification head)
# This new layer's parameters are trainable by default.
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
print(f"Replaced final FC layer with {len(class_names)} output features.")

# 3. Unfreeze specific layers for fine-tuning
# We'll unfreeze 'layer4' (the last block of ResNet's convolutional layers).
# Its parameters will now be included in the optimizer's parameter groups and updated.
print("Unfreezing model.layer4 for fine-tuning...")
for param in model.layer4.parameters():
    param.requires_grad = True

# Optional: Unfreeze more layers if needed.
# For example, to also unfreeze 'layer3' (the second-to-last block):
# print("Unfreezing model.layer3 for fine-tuning...")
# for param in model.layer3.parameters():
#     param.requires_grad = True

# Move the model to the chosen device (GPU if available)
model = model.to(device)

# Define Loss Function and Optimizer
# Pass the calculated class_weights to the criterion to handle class imbalance
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device)) # <--- IMPORTANT: Weighted Loss

# Define Optimizer with different learning rates for different parameter groups.
# This is crucial for effective fine-tuning:
#   - The new classification head (model.fc) can use a higher learning rate.
#   - Unfrozen pre-trained layers (model.layer4, and optionally model.layer3) need a much smaller learning rate
#     to adapt to the new data without destroying their powerful learned features.
optimizer = optim.Adam([
    {'params': model.fc.parameters(), 'lr': 1e-4}, # Learning rate for the classification head
    {'params': model.layer4.parameters(), 'lr': 1e-5}, # Lower LR for the last conv block
    # {'params': model.layer3.parameters(), 'lr': 5e-6}, # Even lower LR if layer3 is unfrozen
], lr=1e-4) # A default fallback LR for any parameters not explicitly grouped

# Learning Rate Scheduler: ReduceLROnPlateau will monitor validation loss
# and reduce the learning rate by a factor of 0.1 if validation loss doesn't improve for 5 epochs.
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)



Classes detected by ImageFolder: ['Bad Weld', 'Defect', 'Good Weld']
Dataset sizes: {'train': 2580, 'valid': 444, 'test': 186}
Using device: cuda
Freezing all base model parameters...
Replaced final FC layer with 3 output features.
Unfreezing model.layer4 for fine-tuning...


In [14]:
num_epochs = 30 # Number of training epochs
best_model_wts_path = "/kaggle/working/best_classification_model.pth" # Path to save the best model
best_acc = 0.0 # Initialize best accuracy for saving the best model

print("\n--- Starting Training ---")
# Iterate through each epoch
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'valid']:
        if phase == 'train':
            model.train()  # Set model to training mode (enables dropout, batchnorm updates)
        else:
            model.eval()   # Set model to evaluate mode (disables dropout, fixes batchnorm)

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data batches
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device) # Move inputs to the device
            labels = labels.to(device) # Move labels to the device

            # Zero the parameter gradients before each batch
            optimizer.zero_grad()

            # Forward pass: track gradients only if in training phase
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)        # Get model predictions
                _, preds = torch.max(outputs, 1) # Get the predicted class (index of max logit)
                loss = criterion(outputs, labels) # Calculate the loss

                # Backward pass + optimize only if in training phase
                if phase == 'train':
                    loss.backward()  # Compute gradients
                    optimizer.step() # Update model parameters

            # Statistics for the current phase
            running_loss += loss.item() * inputs.size(0) # Accumulate batch loss
            running_corrects += torch.sum(preds == labels.data) # Accumulate correct predictions

        # Calculate epoch-level loss and accuracy
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Step the learning rate scheduler if in validation phase
        if phase == 'valid':
            scheduler.step(epoch_loss) # Update scheduler based on validation loss

            # Save the model if it's the best performing on the validation set
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), best_model_wts_path)
                print(f"New best validation accuracy: {best_acc:.4f}. Model saved to {best_model_wts_path}")

print("\n--- Training Complete ---")
print(f"Best validation accuracy achieved: {best_acc:.4f}")

# --- Load Best Model and Evaluate on Test Set ---
print("\n--- Evaluating Best Model on Test Set ---")
# Load the weights of the best performing model from validation
model.load_state_dict(torch.load(best_model_wts_path)) 
model.eval() # Set model to evaluation mode

test_running_corrects = 0
test_total = 0

# Disable gradient calculation for evaluation to save memory and speed up computation
with torch.no_grad(): 
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        test_running_corrects += torch.sum(preds == labels.data)
        test_total += labels.size(0)

test_acc = test_running_corrects.double() / test_total
print(f'Test Accuracy of the best model: {test_acc:.4f}')




--- Starting Training ---
Epoch 1/30
----------
train Loss: 1.1146 Acc: 0.3760
valid Loss: 1.0831 Acc: 0.4167
New best validation accuracy: 0.4167. Model saved to /kaggle/working/best_classification_model.pth
Epoch 2/30
----------
train Loss: 1.0119 Acc: 0.4403
valid Loss: 1.0259 Acc: 0.4572
New best validation accuracy: 0.4572. Model saved to /kaggle/working/best_classification_model.pth
Epoch 3/30
----------
train Loss: 0.9748 Acc: 0.4733
valid Loss: 1.0059 Acc: 0.4752
New best validation accuracy: 0.4752. Model saved to /kaggle/working/best_classification_model.pth
Epoch 4/30
----------
train Loss: 0.9484 Acc: 0.5000
valid Loss: 0.9704 Acc: 0.4887
New best validation accuracy: 0.4887. Model saved to /kaggle/working/best_classification_model.pth
Epoch 5/30
----------
train Loss: 0.9187 Acc: 0.5120
valid Loss: 0.9524 Acc: 0.4977
New best validation accuracy: 0.4977. Model saved to /kaggle/working/best_classification_model.pth
Epoch 6/30
----------
train Loss: 0.9078 Acc: 0.5229
valid